In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
img_width, img_height = 256, 256
train_data_dir = "Final Dataset/train"
validation_data_dir = "Final Dataset/val"
test_data_dir = "Final Dataset/test"
nb_train_samples = 360
nb_validation_samples = 16 
nb_test_samples = 20
batch_size = 16
epochs = 50

In [3]:
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [5]:
for layer in model.layers[:5]:
    layer.trainable = False

In [6]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [7]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [8]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [9]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
width_shift_range = 0.05,
height_shift_range=0.05,
rotation_range=15)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
width_shift_range = 0.05,
height_shift_range=0.05,
rotation_range=15)

In [10]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 704 images belonging to 2 classes.


In [11]:
validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 16 images belonging to 2 classes.


In [12]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [13]:
# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
D:\Programs\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=22, validation_steps=16, epochs=50, callbacks=[<keras.ca...)`
  


Epoch 1/50
22/22 [==============================] - 32s 1s/step - loss: 0.7359 - acc: 0.5114 - val_loss: 0.6948 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to vgg16_1.h5
Epoch 2/50
22/22 [==============================] - 15s 686ms/step - loss: 0.7329 - acc: 0.5625 - val_loss: 0.7700 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/50
22/22 [==============================] - 15s 673ms/step - loss: 0.7142 - acc: 0.5284 - val_loss: 0.6627 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/50
22/22 [==============================] - 15s 673ms/step - loss: 0.6775 - acc: 0.5739 - val_loss: 0.6417 - val_acc: 0.7148

Epoch 00004: val_acc improved from 0.50000 to 0.71484, saving model to vgg16_1.h5
Epoch 5/50
22/22 [==============================] - 15s 674ms/step - loss: 0.6475 - acc: 0.6250 - val_loss: 0.6472 - val_acc: 0.6445

Epoch 00005: val_acc did not improve from 0.71484
Epoch 6/50
22/22 [====

In [14]:
test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
width_shift_range = 0.05,
height_shift_range=0.05,
rotation_range=15)

In [15]:
test_generator = test_datagen.flow_from_directory(
test_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 20 images belonging to 2 classes.


In [16]:
score = model_final.evaluate_generator(test_generator, steps=20)

In [17]:
print("Accuracy of model on test data : ", score[1] * 100)

Accuracy of model on test data :  95.5
